In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import torch
from sklearn.cluster import KMeans
import os
from tqdm import tqdm
import ast
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  # 파일로 저장하는 백엔드 사용

# 파일 경로 설정
test_path = './processed_test/'
model_path = './new_model_save/time_series_cluster_models.pth'
error_path = './result/reconstruction_errors.csv'
output_path = './cluster_submission_2.csv'

In [ ]:
# 모델 클래스 정의
class TimeSeriesSensorAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, latent_dim=32, hidden_dim=256, window_size=1440, weight_decay=1e-4, sensor_weights=None):
        super(TimeSeriesSensorAutoencoder, self).__init__()
        self.weight_decay = weight_decay
        
        if sensor_weights is None:
            self.sensor_weights = {
                'Q': 1.0,
                'M': 0.5,
                'P': 2.0
            }
        else:
            self.sensor_weights = sensor_weights
            
        self.encoder_lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )
        
        self.encoder_fc = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(hidden_dim, latent_dim),
            torch.nn.BatchNorm1d(latent_dim),
            torch.nn.ReLU()
        )
        
        self.decoder_fc = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        
        self.decoder_lstm = torch.nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )
        
        self.output_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        batch_size = x.size(0)
        device = x.device
        
        lstm_out, _ = self.encoder_lstm(x)
        last_hidden = lstm_out[:, -1, :]
        z = self.encoder_fc(last_hidden)
        
        decoded = self.decoder_fc(z)
        decoded = decoded.unsqueeze(1).repeat(1, x.size(1), 1)
        decoded, _ = self.decoder_lstm(decoded)
        reconstructed = self.output_layer(decoded[:, -1, :])
        
        return z, reconstructed

In [ ]:
def visualize_cluster_error_distribution(errors_df, cluster_info, cluster_stats):
    """클러스터별 재구성 오류 분포 시각화 (TEST_C, TEST_D 분리)"""
    # 각 클러스터의 오류 데이터 수집 (TEST_C, TEST_D 분리)
    cluster_errors = {
        'TEST_C': {cluster: [] for cluster in range(len(cluster_stats))},
        'TEST_D': {cluster: [] for cluster in range(len(cluster_stats))}
    }
    
    for file_id, errors, clusters in zip(
        errors_df['ID'], 
        errors_df['error_list'].apply(ast.literal_eval), 
        [cluster_info[file_id] for file_id in errors_df['ID']]
    ):
        group = 'TEST_C' if file_id.startswith('TEST_C') else 'TEST_D'
        for error, sensor_cluster in zip(errors, clusters):
            if sensor_cluster != -1:
                cluster_errors[group][sensor_cluster].append(error)
    
    # 각 그룹(TEST_C, TEST_D)에 대해 히스토그램 생성
    for group in ['TEST_C', 'TEST_D']:
        plt.figure(figsize=(20, 12))
        for cluster, errors in cluster_errors[group].items():
            if not errors:
                continue
            
            plt.subplot(2, 3, cluster + 1)
            plt.hist(errors, bins=50, edgecolor='black', alpha=0.7)
            plt.title(f'{group} Cluster {cluster} Reconstruction Error Distribution')
            plt.xlabel('Reconstruction Error')
            plt.ylabel('Frequency')
            
            # 클러스터의 threshold 표시
            threshold = cluster_stats[cluster]['threshold']
            plt.axvline(x=threshold, color='r', linestyle='--', 
                        label=f'Original Threshold: {threshold:.4f}')
            plt.legend()
        
        plt.tight_layout()
        plt.savefig(f'./{group}_cluster_error_distribution.png')
        plt.close()

    # 박스 플롯 (TEST_C, TEST_D 분리)
    for group in ['TEST_C', 'TEST_D']:
        plt.figure(figsize=(15, 8))
        error_data = [errors for errors in cluster_errors[group].values() if errors]
        cluster_labels = [f'Cluster {i}' for i, errors in cluster_errors[group].items() if errors]
        
        plt.boxplot(error_data, tick_labels=cluster_labels)
        plt.title(f'{group} Reconstruction Error Distribution by Cluster')
        plt.ylabel('Reconstruction Error')
        plt.xlabel('Cluster')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.savefig(f'./{group}_cluster_error_boxplot.png')
        plt.close()

    # 추가 통계 정보 출력 (TEST_C, TEST_D 분리)
    for group in ['TEST_C', 'TEST_D']:
        print(f"\n{group} 클러스터별 재구성 오류 통계:")
        for cluster, errors in cluster_errors[group].items():
            if errors:
                print(f"Cluster {cluster}:")
                print(f"  평균 오류: {np.mean(errors):.4f}")
                print(f"  표준편차: {np.std(errors):.4f}")
                print(f"  최대 오류: {np.max(errors):.4f}")
                print(f"  최소 오류: {np.min(errors):.4f}")

# 나머지 기존 함수들 (이전 코드와 동일)
def get_cluster_for_p_sensor(df, p_sensor, kmeans):
    """P센서의 원본 데이터 평균으로 클러스터 할당"""
    p_mean = np.mean(df[p_sensor].values)  # 정규화하지 않은 원본 값의 평균
    return kmeans.predict([[p_mean]])[0]

def process_test_files(kmeans):
    """테스트 파일들의 P센서 클러스터 확인"""
    file_list = sorted(glob(os.path.join(test_path, '*.csv')))
    cluster_info = {}
    cluster_counts = {i: 0 for i in range(kmeans.n_clusters)}  # 클러스터별 카운트
    
    for file_path in tqdm(file_list, desc="Processing files"):
        df = pd.read_csv(file_path)
        file_name = os.path.basename(file_path).split('.')[0]
        p_sensors = [col for col in df.columns if col.startswith('P') and not col.endswith('_flag')]
        
        clusters = []
        for p_sensor in p_sensors:
            cluster = get_cluster_for_p_sensor(df, p_sensor, kmeans)
            clusters.append(cluster)
            cluster_counts[cluster] += 1  # 클러스터별 카운트 증가
            
        cluster_info[file_name] = clusters
    
    # 클러스터 분포 출력
    print("\nCluster Distribution:")
    for cluster, count in cluster_counts.items():
        print(f"Cluster {cluster}: {count} sensors")
    
    return cluster_info

def recommend_cluster_search_ranges(cluster_stats):
    """사용자가 직접 클러스터별 threshold scale 탐색 범위 설정"""
    search_ranges = {}
    
    # 클러스터 threshold 정보 출력
    print("\n클러스터별 원본 Threshold 정보:")
    for cluster, stats in cluster_stats.items():
        original_threshold = stats['threshold']
        print(f"Cluster {cluster}: Original Threshold = {original_threshold:.4f}")
    
    # 사용자 입력 안내
    print("\n각 클러스터의 threshold scale 탐색 범위를 설정해주세요.")
    print("형식: 클러스터 번호 최소값 최대값 (예: 0 5 7)")
    print("모든 입력이 완료되면 'done'을 입력하세요.")
    
    while True:
        user_input = input("입력: ").strip()
        
        if user_input.lower() == 'done':
            break
        
        try:
            cluster, left, right = map(float, user_input.split())
            cluster = int(cluster)
            search_ranges[cluster] = (left, right)
        except ValueError:
            print("잘못된 입력입니다. 다시 입력해주세요.")
    
    # 입력되지 않은 클러스터는 기본값 사용
    for cluster in range(len(cluster_stats)):
        if cluster not in search_ranges:
            search_ranges[cluster] = (13, 14)
    
    print("\n최종 Threshold Scale 탐색 범위:")
    for cluster, (left, right) in search_ranges.items():
        print(f"Cluster {cluster}: Scale 탐색 범위 = ({left}, {right})")
    
    return search_ranges

def get_cluster_for_p_sensor(df, p_sensor, kmeans):
    """P센서의 원본 데이터 평균으로 클러스터 할당"""
    p_mean = np.mean(df[p_sensor].values)  # 정규화하지 않은 원본 값의 평균
    return kmeans.predict([[p_mean]])[0]

def process_test_files(kmeans):
    """테스트 파일들의 P센서 클러스터 확인"""
    file_list = sorted(glob(os.path.join(test_path, '*.csv')))
    cluster_info = {}
    cluster_counts = {i: 0 for i in range(kmeans.n_clusters)}  # 클러스터별 카운트
    
    for file_path in tqdm(file_list, desc="Processing files"):
        df = pd.read_csv(file_path)
        file_name = os.path.basename(file_path).split('.')[0]
        p_sensors = [col for col in df.columns if col.startswith('P') and not col.endswith('_flag')]
        
        clusters = []
        for p_sensor in p_sensors:
            cluster = get_cluster_for_p_sensor(df, p_sensor, kmeans)
            clusters.append(cluster)
            cluster_counts[cluster] += 1  # 클러스터별 카운트 증가
            
        cluster_info[file_name] = clusters
    
    # 클러스터 분포 출력
    print("\nCluster Distribution:")
    for cluster, count in cluster_counts.items():
        print(f"Cluster {cluster}: {count} sensors")
    
    return cluster_info

def count_valid_samples(submission_df):
    """유효 샘플(적어도 하나의 이상이 감지된 파일) 수를 계산"""
    valid_count = 0
    for flags in submission_df['flag_list']:
        flags_list = ast.literal_eval(flags)
        if sum(flags_list) > 0:  # 하나라도 1이 있으면 유효 샘플
            valid_count += 1
    return valid_count

def create_submission_with_custom_threshold(errors_df, cluster_info, cluster_stats, custom_threshold):
    """특정 threshold로 결과 생성"""
    results = []
    
    for _, row in errors_df.iterrows():
        file_id = row['ID']
        errors = ast.literal_eval(row['error_list'])
        clusters = cluster_info[file_id]
        
        anomaly_flags = []
        for error, cluster in zip(errors, clusters):
            # 클러스터가 -1인 경우 이상 감지를 하지 않음
            if cluster == -1:
                anomaly_flags.append(0)
            else:
                is_anomaly = 1 if error > custom_threshold else 0
                anomaly_flags.append(is_anomaly)
        
        results.append({
            'ID': file_id,
            'flag_list': str(anomaly_flags)
        })
    
    return pd.DataFrame(results)

def find_optimal_threshold_scale_by_cluster(
    errors_df, 
    cluster_info, 
    cluster_stats, 
    custom_thresholds
):
    """클러스터별로 설정된 threshold 적용"""
    cluster_scales = {}
    
    # TEST_C와 TEST_D 그룹 분리
    test_c_files = errors_df[errors_df['ID'].str.startswith('TEST_C')]['ID'].tolist()
    test_d_files = errors_df[errors_df['ID'].str.startswith('TEST_D')]['ID'].tolist()
    
    # 그룹별 처리
    for group, files in [('TEST_C', test_c_files), ('TEST_D', test_d_files)]:
        # 현재 그룹의 데이터만 필터링
        group_errors_df = errors_df[errors_df['ID'].isin(files)]
        group_cluster_info = {k: v for k, v in cluster_info.items() if k in files}
        
        # 유효한 총 클러스터 목록 추출
        used_clusters = set(cluster for clusters in group_cluster_info.values() for cluster in clusters if cluster != -1)
        
        for cluster in used_clusters:
            # 현재 클러스터의 데이터만 포함하는 cluster_info 생성
            cluster_specific_info = {
                file_id: [c if c == cluster else -1 for c in clusters]
                for file_id, clusters in group_cluster_info.items()
            }
            
            # 사용자 정의 threshold 사용
            custom_threshold = custom_thresholds[group].get(cluster, cluster_stats[cluster]['threshold'])
            
            # 서브밋 생성
            submission_df = create_submission_with_custom_threshold(
                group_errors_df, 
                cluster_specific_info, 
                cluster_stats, 
                custom_threshold
            )
            
            # 유효 샘플 수 계산
            valid_count = count_valid_samples(submission_df)
            
            # 기존 코드와 호환되는 형식으로 저장
            cluster_scales[cluster] = {
                'scale': custom_threshold / cluster_stats[cluster]['threshold'],  # scale 계산
                'valid_count': valid_count,
                'threshold': custom_threshold
            }
    
    return cluster_scales

def create_final_submission_by_cluster(
    errors_df, 
    cluster_info, 
    cluster_stats, 
    cluster_scales
):
    results = []
    
    for _, row in errors_df.iterrows():
        file_id = row['ID']
        errors = ast.literal_eval(row['error_list'])
        clusters = cluster_info[file_id]
        
        anomaly_flags = []
        for error, cluster in zip(errors, clusters):
            # 클러스터가 -1인 경우 이상 감지를 하지 않음
            if cluster == -1:
                anomaly_flags.append(0)
            else:
                # 해당 클러스터의 threshold 가져오기
                threshold = cluster_scales[cluster]['threshold']
                is_anomaly = 1 if error > threshold else 0
                anomaly_flags.append(is_anomaly)
        
        results.append({
            'ID': file_id,
            'flag_list': str(anomaly_flags)
        })
    
    return pd.DataFrame(results)

def recommend_cluster_search_ranges(cluster_stats):
    """TEST_C와 TEST_D별로 클러스터별 threshold를 하이퍼파라미터로 설정"""
    thresholds = {
        'TEST_C': {
            3: 50,    # TEST_C Cluster 3의 threshold
            5: 50    # TEST_C Cluster 5의 threshold
        },
        'TEST_D': {
            2: 9.493,      # TEST_D Cluster 2의 threshold
            4: 4.54,      # TEST_D Cluster 4의 threshold
            5: 2.714       # TEST_D Cluster 5의 threshold
        }
    }
    
    # 클러스터 threshold 정보 출력
    print("\n클러스터별 Threshold 설정:")
    for group in ['TEST_C', 'TEST_D']:
        print(f"\n{group} 그룹:")
        for cluster, threshold in thresholds[group].items():
            original_threshold = cluster_stats[cluster]['threshold']
            print(f"Cluster {cluster}: 원본 Threshold = {original_threshold:.4f}, 설정된 Threshold = {threshold:.4f}")
    
    return thresholds

def print_detailed_cluster_info(cluster_scales, cluster_stats):
    """클러스터별 상세 정보 출력"""
    print("\n상세 클러스터 정보:")
    for cluster, info in cluster_scales.items():
        original_threshold = cluster_stats[cluster]['threshold']
        adjusted_threshold = original_threshold * info['scale']
        print(f"Cluster {cluster}:")
        print(f"  원본 Threshold: {original_threshold:.4f}")
        print(f"  Scale: {info['scale']:.4f}")
        print(f"  조정된 Threshold: {adjusted_threshold:.4f}")
        print(f"  유효 샘플 수: {info['valid_count']}")

def calculate_performance_metrics(final_submission):
    """이상 탐지 성능에 대한 상세 메트릭 계산"""
    total_anomalies = 0
    total_sensors = 0
    test_c_anomalies = 0
    test_d_anomalies = 0

    for row in final_submission.itertuples():
        file_id = row.ID
        flags_list = ast.literal_eval(row.flag_list)
        
        file_anomalies = sum(flags_list)
        total_anomalies += file_anomalies
        total_sensors += len(flags_list)
        
        if file_id.startswith('TEST_C'):
            test_c_anomalies += file_anomalies
        else:
            test_d_anomalies += file_anomalies

    print("\n성능 평가:")
    print(f"전체 이상 감지 비율: {total_anomalies / total_sensors * 100:.2f}%")
    print(f"전체 센서 중 이상 센서 비율: {total_anomalies} / {total_sensors}")
    print(f"TEST_C 이상 감지 비율: {test_c_anomalies / sum(1 for row in final_submission.itertuples() if row.ID.startswith('TEST_C')) * 100:.2f}%")
    print(f"TEST_D 이상 감지 비율: {test_d_anomalies / sum(1 for row in final_submission.itertuples() if row.ID.startswith('TEST_D')) * 100:.2f}%")
    
    return {
        'total_anomaly_ratio': total_anomalies / total_sensors * 100,
        'total_anomalies': total_anomalies,
        'total_sensors': total_sensors,
        'test_c_anomaly_ratio': test_c_anomalies / sum(1 for row in final_submission.itertuples() if row.ID.startswith('TEST_C')) * 100,
        'test_d_anomaly_ratio': test_d_anomalies / sum(1 for row in final_submission.itertuples() if row.ID.startswith('TEST_D')) * 100
    }

def analyze_cluster_anomalies(errors_df, cluster_info, cluster_stats, cluster_scales):
    """클러스터별 이상 센서 상세 분석"""
    cluster_anomaly_info = {}
    
    for cluster in range(len(cluster_stats)):
        # 해당 클러스터의 모든 센서 추출
        cluster_sensors = [
            (file_id, errors, clusters) 
            for file_id, errors, clusters in zip(
                errors_df['ID'], 
                errors_df['error_list'].apply(ast.literal_eval), 
                [cluster_info[file_id] for file_id in errors_df['ID']]
            )
            if cluster in clusters
        ]
        
        # 클러스터별 이상 감지 분석
        total_sensors = len(cluster_sensors)
        
        # 전체 센서 수가 0이면 해당 클러스터 제외
        if total_sensors == 0:
            continue
        
        anomaly_sensors = 0
        max_error = 0
        min_error = float('inf')
        errors_list = []
        
        for file_id, errors, clusters in cluster_sensors:
            for error, sensor_cluster in zip(errors, clusters):
                if sensor_cluster == cluster:
                    errors_list.append(error)
                    
                    if error > cluster_stats[cluster]['threshold']:
                        anomaly_sensors += 1
                    
                    max_error = max(max_error, error)
                    min_error = min(min_error, error)
        
        cluster_anomaly_info[cluster] = {
            'total_sensors': total_sensors,
            'anomaly_sensors': anomaly_sensors,
            'anomaly_ratio': anomaly_sensors / total_sensors * 100,
            'max_error': max_error,
            'min_error': min_error,
            'mean_error': np.mean(errors_list),
            'scale': cluster_scales[cluster]['scale'] if cluster in cluster_scales else None
        }
    
    # 결과 출력
    print("\n클러스터별 상세 이상 감지 분석:")
    for cluster, info in cluster_anomaly_info.items():
        original_threshold = cluster_stats[cluster]['threshold']
        adjusted_threshold = original_threshold * info['scale'] if info['scale'] is not None else None
        
        print(f"Cluster {cluster}:")
        print(f"  전체 센서 수: {info['total_sensors']}")
        print(f"  이상 센서 수: {info['anomaly_sensors']}")
        print(f"  이상 센서 비율: {info['anomaly_ratio']:.2f}%")
        print(f"  원본 Threshold: {original_threshold:.4f}")
        print(f"  Scale: {info['scale']:.4f}" if info['scale'] is not None else "  Scale: N/A")
        print(f"  조정된 Threshold: {adjusted_threshold:.4f}" if adjusted_threshold is not None else "  조정된 Threshold: N/A")
        print(f"  평균 에러: {info['mean_error']:.4f}")
        print(f"  최대 에러: {info['max_error']:.4f}")
        print(f"  최소 에러: {info['min_error']:.4f}")
    
    return cluster_anomaly_info

In [ ]:
checkpoint = torch.load(model_path)
kmeans = checkpoint['kmeans']
cluster_stats = checkpoint['cluster_stats']

In [ ]:
errors_df = pd.read_csv(error_path)
cluster_info = process_test_files(kmeans)
print("\nFinding optimal threshold scale...")

In [ ]:
# 클러스터별 threshold 설정
custom_thresholds = recommend_cluster_search_ranges(cluster_stats)

# 메인 스크립트에서 함수 호출 수정
cluster_scales = find_optimal_threshold_scale_by_cluster(
    errors_df, 
    cluster_info, 
    cluster_stats, 
    custom_thresholds
)

# 상세 클러스터 정보 출력
print_detailed_cluster_info(cluster_scales, cluster_stats)

# 클러스터 이상 감지 분석 (cluster_scales 전달)
cluster_anomaly_info = analyze_cluster_anomalies(
    errors_df, 
    cluster_info, 
    cluster_stats, 
    cluster_scales  # 이 부분이 중요
)

In [ ]:
final_submission = create_final_submission_by_cluster(
    errors_df, 
    cluster_info, 
    cluster_stats, 
    cluster_scales
)
final_submission.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")

# 성능 메트릭 계산
performance_metrics = calculate_performance_metrics(final_submission)

# 클러스터별 재구성 오류 분포 시각화
visualize_cluster_error_distribution(errors_df, cluster_info, cluster_stats)